I tried to keep this notebook as clean and focused as possible by including only the most relevant lines of code to ensure clarity and ease of understanding. In the markdown cells, I provided explanations of the different algorithms and techniques I experimented with during the process in order to highlight why I ultimately selected a specific approach or model. This way, you can better understand the thought process behind each decision and why certain steps were preferred over others.

In [369]:
import pandas as pd
import numpy as np
import category_encoders as ce
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    train_test_split,
    StratifiedShuffleSplit,
)
from sklearn.impute import IterativeImputer
from sklearn.ensemble import (
    RandomForestRegressor,
    IsolationForest
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)


In [370]:
df = pd.read_csv('data.csv')

C:\Users\valer\AppData\Local\Temp\ipykernel_23484\1507164070.py:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


During the data visualization and exploration phase, I observed that some entries in the dataset are unrealistic or erroneous:

 - Some cars had a manufacture_year before 1886, which is impossible since the first automobile was invented in 1886.

 - Other cars were listed with a manufacture_year after 2025, which is also invalid because 2025 represents the current or near-future year.

 - Regarding price_eur, some entries had prices lower than €500. Based on real-world data and common sense, even the cheapest cars are rarely priced below this value.

 - Additionally, from external research (Google) and from the visualization plots, the maximum realistic car price seems to be around €30,000,000.

To correct these issues and remove outliers that could negatively impact model training, I applied the following filters:

In [371]:
df = df[(df['manufacture_year'] >= 1886) & (df['manufacture_year'] <= 2025)]
df = df[(df['price_eur'] >= 500) & (df['manufacture_year'] <= 30_000_000)]

In [372]:
null_percentage = df.isnull().mean() * 100
null_percentage

maker                  13.833409
model                  31.492740
mileage                 6.789449
manufacture_year        0.000000
engine_displacement    18.451950
engine_power           12.569992
body_type              33.372827
color_slug             93.387076
stk_year               84.371361
transmission           18.221410
door_count             27.397861
seat_count             32.585641
fuel_type              51.764050
date_created            0.000000
date_last_seen          0.000000
price_eur               0.000000
dtype: float64

The columns stk_year and color_slug had high missing rates (84% and 93%, respectively) and were not critical for predicting car prices.
Imputing these values introduced noise, dropping the model’s R² score from 0.82 to 0.60.
Therefore, I removed them to improve model performance and data quality.

In [373]:
df = df.drop('stk_year',axis=1)
df = df.drop('color_slug',axis=1)

I processed the date_created and date_last_seen columns to help the model better understand the car's listing activity over time.
Instead of keeping two separate date columns, I created a single feature (days_since_created) that captures the time difference, making it easier for the model to learn from it.

In [374]:
df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
df['date_last_seen'] = pd.to_datetime(df['date_last_seen'], errors='coerce')
df = df.dropna(subset=['date_created', 'date_last_seen'], how='all')

df['days_since_created'] = (df['date_last_seen'] - df['date_created']).dt.days
df.drop(['date_created', 'date_last_seen'], axis=1, inplace=True)

I separated the numerical and categorical columns to ensure proper preprocessing for each type of data. Categorical columns require encoding because machine learning algorithms cannot directly work with text data.
On the other hand, numerical columns  need scaling. Scaling ensures that all numerical features are on a similar scale, preventing any one feature from dominating due to differences in their value ranges.

In [375]:
categorical_cols = ['maker', 'model', 'body_type', 'transmission', 'fuel_type']
numerical_cols = ['mileage', 'manufacture_year', 'engine_displacement', 
                  'engine_power', 'door_count', 'seat_count',
                  'days_since_created' ]

I decided to use IterativeImputer for imputing missing values in the dataset because it is the most optimal imputation method for this case.

I initially tried using KNNImputer, but it was very slow. It took over an hour to process and still didn't finish, making it impractical for this dataset. On the other hand, SimpleImputer was too basic for the problem because it only fills missing values with the mean of the column, which doesn't account for the relationships between different features.

In [376]:
imputer = IterativeImputer(max_iter=10, random_state=42)
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

Initially, I tried One-Hot Encoding, but it created too many features, significantly increasing computation time, and I had to stop it. I also tested Ordinal Encoding, which is faster but doesn't capture the relationship between categorical variables and the target, resulting in lower accuracy.

The best approach was Target Encoding, which replaces categories with the mean price (price_eur) for each category. This method captures the relationship between the features and target, improving both time efficiency and model accuracy without compromising on performance.

In [377]:
target_enc = TargetEncoder(cols=categorical_cols)
df[categorical_cols] = target_enc.fit_transform(df[categorical_cols], y=df['price_eur'])


One of the biggest challenges with this dataset was the presence of outliers. These outliers severely skewed the model’s predictions, making it worse than simply guessing a random price.

To address this, I used the Isolation Forest algorithm, which is effective at detecting outliers. However, setting the correct contamination parameter was tricky because it determines the proportion of outliers. Initially, I set it to 0.01, but I realized I needed a more precise estimate.

To improve the detection, I used the estimated contamination value based on the model's initial predictions. By adjusting the contamination parameter dynamically, the model was able to better identify and remove outliers, leading to a cleaner dataset.

In [378]:
iso = IsolationForest(contamination=0.01, random_state=42)
iso.fit(df)
scores = iso.decision_function(df)

threshold = np.percentile(scores, 5)
predictions = (scores < threshold).astype(int)
estimated_contamination = predictions.sum() / len(predictions)

iso = IsolationForest(contamination=estimated_contamination, random_state=42)
iso.fit(df)
final_preds = iso.predict(df)

df_cleaned = df[final_preds == 1]

Another significant challenge in this dataset was the enormous number of rows, which led to a lot of noise. This noise caused overfitting and made the model take hours to run. With too many data points, the model becomes overly complex, capturing patterns that don't generalize well, resulting in poor accuracy.

To tackle this, I decided to work with a sample size of 10,000. This reduced the dataset to a manageable size while still retaining sufficient variability for training. The sample size improvement helped the model run faster and reduced the risk of overfitting.

I also applied binning to the price column  to group similar values together. This process divides the continuous target variable into discrete bins to create better balance across categories:

In [379]:
price_bins = np.linspace(df_cleaned['price_eur'].min(), df_cleaned['price_eur'].max(), 10)

df_cleaned.loc[:, 'price_bin'] = pd.cut(df_cleaned['price_eur'], bins=price_bins, labels=False, include_lowest=True)

bin_counts = df_cleaned['price_bin'].value_counts()
valid_bins = bin_counts[bin_counts >= 2].index
df_cleaned = df_cleaned[df_cleaned['price_bin'].isin(valid_bins)]

C:\Users\valer\AppData\Local\Temp\ipykernel_23484\976949412.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.loc[:, 'price_bin'] = pd.cut(df_cleaned['price_eur'], bins=price_bins, labels=False, include_lowest=True)


After binning, I used Stratified Shuffle Split to ensure that the sample had a representative distribution of the price categories. This step is vital for training on a balanced subset that reflects the actual data distribution.

Finally, I split the data into training and testing sets and removed the price_bin column for the training process. This allowed me to train the model on a clean, representative sample, improving both efficiency and model performance.

In [380]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=10000, random_state=42)

for train_index, test_index in splitter.split(df_cleaned, df_cleaned['price_bin']):
    stratified_sample = df_cleaned.iloc[test_index]

stratified_sample = stratified_sample.drop(columns=['price_bin'])

y = stratified_sample['price_eur']
X = stratified_sample.drop(columns=['price_eur'])

By applying log, I transform the target variable into a form that's easier for the model to predict, reduces the influence of extreme values, and improves the overall performance of the model.

In [381]:
y_log = np.log1p(y)

Standardizing the features ensures all input variables are on the same scale, preventing any one feature from dominating the learning process due to its larger magnitude.

In [382]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [383]:
X_train, X_test, y_train_log, y_test_log = train_test_split(X_scaled, y_log, test_size=0.3, random_state=42)

After testing multiple models, I chose the RandomForestRegressor due to its superior accuracy compared to other models such as XGBoost, SVR, and Linear Regression on this datase

In [387]:
rf = RandomForestRegressor(random_state=42)

rf.fit(X_train, y_train_log)

RandomForestRegressor(random_state=42)

R² score of 0.8267 suggests that the Random Forest model has a strong fit to the data and is able to explain a large portion of the variance in car prices.

Low MSE and MAE values demonstrate that the model is generating relatively accurate price predictions, with the MAE indicating that the typical error is within a reasonable range.

In [391]:
y_pred_log = rf.predict(X_test)

y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test_log)

print(f"Optimized R² Score: {r2_score(y_true, y_pred):.4f}")
print(f"Optimized MSE: {mean_squared_error(y_true, y_pred):.2f}")
print(f"Optimized MAE: {mean_absolute_error(y_true, y_pred):.2f}")


Optimized R² Score: 0.8267
Optimized MSE: 19973377.14
Optimized MAE: 2296.08
